# FastVLM Training with Qwen3-0.6B on Kaggle

This notebook trains FastVLM (Vision Language Model) using:
- **LLM Backbone**: Qwen3-0.6B
- **Vision Encoder**: FastViTHD (MobileCLIP)
- **Dataset**: 5CD-AI/Viet-multimodal-open-r1-8k-verified
- **Output**: Push to HuggingFace: beyoru/Belle-VLM

## Requirements
- Kaggle GPU (P100/T4/A100)
- ~16GB VRAM for full training, or use QLoRA for less

## 1. Install Dependencies

In [ ]:
# Install required packages
!pip install -q transformers>=4.51.0,<5.0.0
!pip install -q torch>=2.1.0 torchvision>=0.16.0
!pip install -q accelerate>=0.26.0 peft>=0.10.0
!pip install -q bitsandbytes>=0.43.0
!pip install -q datasets pillow einops timm>=0.9.0
!pip install -q deepspeed sentencepiece
!pip install -q huggingface_hub

In [ ]:
# Clone the FastVLM repository
!git clone https://github.com/Hert4/ml-fastvlm-v2.git
%cd ml-fastvlm-v2

In [ ]:
# Verify Qwen3 support
!python test_qwen3_load.py

## 2. Setup & Configuration

In [ ]:
import os
import json
import torch
from pathlib import Path

# Configuration
CONFIG = {
    # Model
    "llm_model": "Qwen/Qwen3-0.6B",
    "vision_tower": "mobileclip_l_384",
    
    # Dataset
    "dataset_name": "5CD-AI/Viet-multimodal-open-r1-8k-verified",
    "image_column": "image",
    "question_column": "vi_problem",
    "answer_column": "vi_solution",
    
    # Training - USE STEPS INSTEAD OF EPOCHS
    "output_dir": "./outputs/fastvlm-qwen3-0.6b-vietnamese",
    "max_steps": 2000,  # <-- CHANGE THIS (instead of num_train_epochs)
    "per_device_train_batch_size": 2,
    "gradient_accumulation_steps": 8,
    "learning_rate": 2e-5,
    "warmup_steps": 100,  # <-- Changed from warmup_ratio
    "lr_scheduler_type": "cosine",
    "bf16": True,
    "model_max_length": 2048,
    
    # LoRA
    "use_lora": True,
    "lora_r": 64,
    "lora_alpha": 128,
    "lora_dropout": 0.05,
    
    # Save checkpoints
    "save_steps": 500,
    "save_total_limit": 2,
    
    # HuggingFace
    "hf_repo": "beyoru/Belle-VLM",
    "hf_token": os.environ.get("HF_TOKEN", ""),
}

os.makedirs(CONFIG["output_dir"], exist_ok=True)

print("Configuration:")
for k, v in CONFIG.items():
    if k != "hf_token":
        print(f"  {k}: {v}")

# Estimate training time
effective_batch = CONFIG["per_device_train_batch_size"] * CONFIG["gradient_accumulation_steps"]
print(f"\n📊 Training info:")
print(f"  Effective batch size: {effective_batch}")
print(f"  Total steps: {CONFIG['max_steps']}")
print(f"  Checkpoints at: {list(range(CONFIG['save_steps'], CONFIG['max_steps']+1, CONFIG['save_steps']))}")

In [ ]:
# Login to HuggingFace
from huggingface_hub import login

if CONFIG["hf_token"]:
    login(token=CONFIG["hf_token"])
    print("✅ Logged in to HuggingFace!")
else:
    print("⚠️ HF_TOKEN not set. To push to HuggingFace, set your token:")
    print("   1. Go to Kaggle notebook settings")
    print("   2. Add Secret: Name='HF_TOKEN', Value='your_hf_token'")
    print("   3. Enable 'Add-ons' > 'Secrets' in your notebook")

## 3. Load and Prepare Dataset

In [ ]:
from datasets import load_dataset
from PIL import Image
import io

# Load dataset
print(f"Loading dataset: {CONFIG['dataset_name']}")
dataset = load_dataset(CONFIG["dataset_name"], split="train")

print(f"\nDataset info:")
print(f"  Total samples: {len(dataset)}")
print(f"  Columns: {dataset.column_names}")

# Show sample
sample = dataset[0]
print(f"\nSample data:")
print(f"  vi_problem: {sample['vi_problem'][:200]}...")
print(f"  vi_solution: {sample['vi_solution'][:200]}...")

In [ ]:
# Display sample image
from IPython.display import display

sample_image = sample['image']
if isinstance(sample_image, Image.Image):
    display(sample_image.resize((400, 400)))
else:
    print(f"Image type: {type(sample_image)}")

In [ ]:
import json
import os
from tqdm import tqdm

def convert_dataset_to_llava_format(dataset, output_dir, image_folder):
    """
    Convert dataset to LLaVA training format.
    
    LLaVA format:
    {
        "id": "unique_id",
        "image": "image_filename.jpg",
        "conversations": [
            {"from": "human", "value": "<image>\nQuestion"},
            {"from": "gpt", "value": "Answer"}
        ]
    }
    """
    os.makedirs(image_folder, exist_ok=True)
    os.makedirs(output_dir, exist_ok=True)
    
    llava_data = []
    
    for idx, sample in enumerate(tqdm(dataset, desc="Converting dataset")):
        # Save image
        image_filename = f"{idx:06d}.jpg"
        image_path = os.path.join(image_folder, image_filename)
        
        img = sample['image']
        if isinstance(img, Image.Image):
            # Convert to RGB if necessary
            if img.mode != 'RGB':
                img = img.convert('RGB')
            img.save(image_path, 'JPEG', quality=95)
        
        # Create conversation
        question = sample['vi_problem'].strip()
        answer = sample['vi_solution'].strip()
        
        # Truncate very long solutions to avoid memory issues
        max_answer_len = 4096
        if len(answer) > max_answer_len:
            answer = answer[:max_answer_len] + "..."
        
        llava_sample = {
            "id": str(sample.get('id', idx)),
            "image": image_filename,
            "conversations": [
                {
                    "from": "human",
                    "value": f"<image>\n{question}"
                },
                {
                    "from": "gpt",
                    "value": answer
                }
            ]
        }
        llava_data.append(llava_sample)
    
    # Save JSON
    json_path = os.path.join(output_dir, "train_data.json")
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(llava_data, f, ensure_ascii=False, indent=2)
    
    print(f"\nDataset converted:")
    print(f"  JSON file: {json_path}")
    print(f"  Images: {image_folder}")
    print(f"  Total samples: {len(llava_data)}")
    
    return json_path, image_folder

# Convert dataset
DATA_DIR = "./data"
IMAGE_FOLDER = os.path.join(DATA_DIR, "images")

json_path, image_folder = convert_dataset_to_llava_format(
    dataset, 
    DATA_DIR, 
    IMAGE_FOLDER
)

## 4. Setup Training

In [ ]:
# Training arguments (for reference)
import sys
sys.path.insert(0, '.')

training_args = {
    "model_name_or_path": CONFIG["llm_model"],
    "version": "qwen_3",
    "vision_tower": CONFIG["vision_tower"],
    "mm_projector_type": "mlp2x_gelu",
    "data_path": json_path,
    "image_folder": image_folder,
    "output_dir": CONFIG["output_dir"],
    
    # STEPS CONFIG
    "max_steps": CONFIG["max_steps"],
    "warmup_steps": CONFIG["warmup_steps"],
    "save_steps": CONFIG["save_steps"],
    
    "per_device_train_batch_size": CONFIG["per_device_train_batch_size"],
    "gradient_accumulation_steps": CONFIG["gradient_accumulation_steps"],
    "learning_rate": CONFIG["learning_rate"],
    "lr_scheduler_type": CONFIG["lr_scheduler_type"],
    "bf16": CONFIG["bf16"],
    "lora_enable": CONFIG["use_lora"],
    "lora_r": CONFIG["lora_r"],
    "lora_alpha": CONFIG["lora_alpha"],
}

print("Training config:")
print(f"  Max steps: {CONFIG['max_steps']}")
print(f"  Warmup steps: {CONFIG['warmup_steps']}")
print(f"  Save every: {CONFIG['save_steps']} steps")

In [ ]:
# Alternative: Direct Python training (recommended for Kaggle)

import torch
from transformers import AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Check GPU
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Load tokenizer
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    CONFIG["llm_model"],
    model_max_length=CONFIG["model_max_length"],
    padding_side="right",
    use_fast=False,
)

# Set pad token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"Tokenizer loaded: {CONFIG['llm_model']}")
print(f"Vocab size: {tokenizer.vocab_size}")

In [ ]:
# Load model with QLoRA (4-bit quantization for memory efficiency)
from llava.model.language_model.llava_qwen import LlavaQwen3ForCausalLM, LlavaQwen3Config

# Quantization config for 4-bit
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

print("Loading model...")
model = LlavaQwen3ForCausalLM.from_pretrained(
    CONFIG["llm_model"],
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

print(f"Model loaded!")
print(f"Model type: {model.config.model_type}")

In [ ]:
# Prepare model for k-bit training
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)

# Setup LoRA
lora_config = LoraConfig(
    r=CONFIG["lora_r"],
    lora_alpha=CONFIG["lora_alpha"],
    lora_dropout=CONFIG["lora_dropout"],
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

## 5. Train the Model

In [ ]:
# Run training using the train_qwen.py script
import subprocess
import torch
import os

# Force single GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Check TF32 support
def supports_tf32():
    if torch.cuda.is_available():
        return torch.cuda.get_device_capability()[0] >= 8
    return False

use_tf32 = supports_tf32()
print(f"GPU supports TF32: {use_tf32}")

# Build command - USING MAX_STEPS
cmd = [
    "python", "-m", "llava.train.train_qwen",
    f"--model_name_or_path={CONFIG['llm_model']}",
    "--version=qwen_3",
    f"--data_path={json_path}",
    f"--image_folder={image_folder}",
    f"--vision_tower={CONFIG['vision_tower']}",
    "--mm_projector_type=mlp2x_gelu",
    "--mm_vision_select_layer=-2",
    "--mm_use_im_start_end=False",
    "--mm_use_im_patch_token=False",
    "--image_aspect_ratio=pad",
    f"--output_dir={CONFIG['output_dir']}",
    # USE MAX_STEPS INSTEAD OF EPOCHS
    f"--max_steps={CONFIG['max_steps']}",
    f"--per_device_train_batch_size={CONFIG['per_device_train_batch_size']}",
    f"--gradient_accumulation_steps={CONFIG['gradient_accumulation_steps']}",
    f"--learning_rate={CONFIG['learning_rate']}",
    "--weight_decay=0.0",
    f"--warmup_steps={CONFIG['warmup_steps']}",  # Changed from warmup_ratio
    f"--lr_scheduler_type={CONFIG['lr_scheduler_type']}",
    "--logging_steps=10",
    f"--save_steps={CONFIG['save_steps']}",
    f"--save_total_limit={CONFIG['save_total_limit']}",
    f"--model_max_length={CONFIG['model_max_length']}",
    "--gradient_checkpointing=True",
    "--dataloader_num_workers=4",
    "--lazy_preprocess=True",
    "--bf16=True",
    "--report_to=none",
    # LoRA
    "--lora_enable=True",
    f"--lora_r={CONFIG['lora_r']}",
    f"--lora_alpha={CONFIG['lora_alpha']}",
    f"--lora_dropout={CONFIG['lora_dropout']}",
]

if use_tf32:
    cmd.append("--tf32=True")
else:
    cmd.append("--tf32=False")

print(f"Starting training for {CONFIG['max_steps']} steps...")
print(f"Checkpoints will be saved every {CONFIG['save_steps']} steps")

env = os.environ.copy()
env["CUDA_VISIBLE_DEVICES"] = "0"

process = subprocess.Popen(
    cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    universal_newlines=True,
    bufsize=1,
    env=env,
)

for line in process.stdout:
    print(line, end='')

process.wait()
print(f"\n✅ Training completed with return code: {process.returncode}")

## 6. Save Model Properly (IMPORTANT!)

This section saves the trained model with ALL weights including mm_projector.

In [ ]:
# ============================================
# CHECK TRAINING OUTPUT
# ============================================
import os

OUTPUT_DIR = CONFIG["output_dir"]
print(f"Checking: {OUTPUT_DIR}\n")

# Check for checkpoint folders
checkpoints = []
if os.path.exists(OUTPUT_DIR):
    for item in os.listdir(OUTPUT_DIR):
        item_path = os.path.join(OUTPUT_DIR, item)
        if os.path.isdir(item_path) and "checkpoint" in item:
            checkpoints.append(item_path)

# List all files
all_files = []
for root, dirs, files in os.walk(OUTPUT_DIR):
    for f in files:
        path = os.path.join(root, f)
        size = os.path.getsize(path) / 1024 / 1024
        all_files.append((path, size))
        print(f"  {path} ({size:.2f} MB)")

print(f"\nTotal files: {len(all_files)}")
print(f"Checkpoints found: {checkpoints}")

# Check important files
if len(all_files) == 0:
    print("\n⚠️ OUTPUT DIRECTORY IS EMPTY!")
    print("👉 Bạn cần chạy CELL TRAINING (cell 19) trước!")
    print("👉 Đảm bảo training hoàn thành và save checkpoint")
    TRAINING_DONE = False
else:
    # Find latest checkpoint or root
    if checkpoints:
        LOAD_PATH = sorted(checkpoints)[-1]  # Latest checkpoint
        print(f"\n✅ Will load from checkpoint: {LOAD_PATH}")
    else:
        LOAD_PATH = OUTPUT_DIR
        print(f"\n✅ Will load from: {LOAD_PATH}")
    TRAINING_DONE = True

In [ ]:
# ============================================
# EXTRACT MM_PROJECTOR FROM NON_LORA_TRAINABLES
# ============================================
import torch
import os
import shutil

OUTPUT_DIR = CONFIG["output_dir"]
MERGED_DIR = os.path.join(OUTPUT_DIR, "merged")
os.makedirs(MERGED_DIR, exist_ok=True)

# Load non_lora_trainables.bin (contains mm_projector)
non_lora_path = os.path.join(OUTPUT_DIR, "non_lora_trainables.bin")

if os.path.exists(non_lora_path):
    print(f"Loading: {non_lora_path}")
    non_lora_weights = torch.load(non_lora_path, map_location="cpu")
    
    print(f"\nKeys in non_lora_trainables.bin:")
    for k in non_lora_weights.keys():
        print(f"  {k}: {non_lora_weights[k].shape}")
    
    # Extract mm_projector weights
    mm_projector_weights = {}
    for k, v in non_lora_weights.items():
        if 'mm_projector' in k:
            # Clean key name
            clean_key = k.replace('base_model.model.model.', 'model.')
            clean_key = clean_key.replace('model.model.', 'model.')
            mm_projector_weights[clean_key] = v
            print(f"  ✅ Extracted: {clean_key}")
    
    # Save mm_projector.bin
    mm_projector_path = os.path.join(MERGED_DIR, "mm_projector.bin")
    torch.save(mm_projector_weights, mm_projector_path)
    print(f"\n✅ Saved mm_projector to: {mm_projector_path}")
    print(f"   Size: {os.path.getsize(mm_projector_path) / 1024 / 1024:.2f} MB")
else:
    print(f"❌ File not found: {non_lora_path}")

In [ ]:
# ============================================
# MERGE LORA + MM_PROJECTOR INTO ONE MODEL
# ============================================
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Load base model
print("Loading base Qwen3 model...")
base_model = AutoModelForCausalLM.from_pretrained(
    CONFIG["llm_model"],
    torch_dtype=torch.float16,
    device_map="cpu",  # Load to CPU for merging
    trust_remote_code=True,
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(CONFIG["llm_model"], trust_remote_code=True)
tokenizer.save_pretrained(MERGED_DIR)

# Load and merge LoRA
print("Loading LoRA adapter...")
model = PeftModel.from_pretrained(base_model, OUTPUT_DIR)

print("Merging LoRA weights...")
model = model.merge_and_unload()

# Load mm_projector from non_lora_trainables.bin
print("\nLoading mm_projector weights...")
non_lora_path = os.path.join(OUTPUT_DIR, "non_lora_trainables.bin")
non_lora_weights = torch.load(non_lora_path, map_location="cpu")

# Get model state dict
state_dict = model.state_dict()

# Add mm_projector weights to state dict
mm_count = 0
for k, v in non_lora_weights.items():
    if 'mm_projector' in k:
        # Clean key: remove prefixes
        clean_key = k
        for prefix in ['base_model.model.', 'model.']:
            if clean_key.startswith(prefix):
                clean_key = clean_key[len(prefix):]
        
        # Add "model." prefix for LlavaQwen3 structure
        if not clean_key.startswith('model.'):
            clean_key = 'model.' + clean_key
        
        state_dict[clean_key] = v.to(torch.float16)
        print(f"  ✅ Added: {clean_key} {v.shape}")
        mm_count += 1

print(f"\nAdded {mm_count} mm_projector tensors")

# Save merged model with mm_projector included
print("\nSaving complete model...")
model.save_pretrained(MERGED_DIR, state_dict=state_dict, safe_serialization=True)

print(f"\n✅ Model saved to: {MERGED_DIR}")
print("   mm_projector is NOW INCLUDED in model.safetensors!")

In [ ]:
# ============================================
# COPY CONFIG AND VERIFY
# ============================================
import shutil
import json

# Copy config.json from training output (has mm_vision_tower settings)
src_config = os.path.join(OUTPUT_DIR, "config.json")
dst_config = os.path.join(MERGED_DIR, "config.json")

if os.path.exists(src_config):
    # Load and update config
    with open(src_config, 'r') as f:
        config_data = json.load(f)
    
    # Add auto_map for HuggingFace
    config_data["auto_map"] = {
        "AutoConfig": "configuration_llava_qwen.LlavaQwen3Config",
        "AutoModelForCausalLM": "modeling_llava_qwen.LlavaQwen3ForCausalLM"
    }
    config_data["architectures"] = ["LlavaQwen3ForCausalLM"]
    
    with open(dst_config, 'w') as f:
        json.dump(config_data, f, indent=2)
    
    print("✅ Updated config.json with auto_map")

# Verify merged directory
print(f"\n📁 Files in {MERGED_DIR}:")
for f in sorted(os.listdir(MERGED_DIR)):
    size = os.path.getsize(os.path.join(MERGED_DIR, f)) / 1024 / 1024
    print(f"  {f} ({size:.2f} MB)")

# Check important files
print("\n✅ Verification:")
for f in ["model.safetensors", "mm_projector.bin", "config.json", "tokenizer.json"]:
    path = os.path.join(MERGED_DIR, f)
    status = "✅" if os.path.exists(path) else "❌"
    print(f"  {status} {f}")

## 7. Push to HuggingFace

In [ ]:
from huggingface_hub import HfApi, create_repo

# Initialize API
api = HfApi(token=CONFIG["hf_token"])

# Create repo if not exists
try:
    create_repo(
        repo_id=CONFIG["hf_repo"],
        repo_type="model",
        exist_ok=True,
        token=CONFIG["hf_token"]
    )
    print(f"Repository ready: {CONFIG['hf_repo']}")
except Exception as e:
    print(f"Repo exists or error: {e}")

In [ ]:
# Create model card
model_card = f"""---
license: apache-2.0
language:
- vi
- en
tags:
- vision-language-model
- vlm
- qwen3
- fastvlm
- vietnamese
base_model: {CONFIG['llm_model']}
datasets:
- {CONFIG['dataset_name']}
---

# Belle-VLM: Vietnamese Vision Language Model

## Model Description

Belle-VLM is a Vision Language Model trained for Vietnamese multimodal reasoning tasks.

### Architecture
- **LLM Backbone**: Qwen3-0.6B
- **Vision Encoder**: FastViTHD (MobileCLIP)
- **Projector**: MLP 2-layer (3072 -> 1024)

### Training
- **Dataset**: 5CD-AI/Viet-multimodal-open-r1-8k-verified
- **Method**: LoRA fine-tuning
- **Steps**: {CONFIG['max_steps']}
- **Learning Rate**: {CONFIG['learning_rate']}

## Usage

```python
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model = AutoModelForCausalLM.from_pretrained(
    "{CONFIG['hf_repo']}",
    trust_remote_code=True,
    torch_dtype=torch.float16,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("{CONFIG['hf_repo']}", trust_remote_code=True)
```

## Training Details

| Parameter | Value |
|-----------|-------|
| Base Model | {CONFIG['llm_model']} |
| Vision Tower | {CONFIG['vision_tower']} |
| LoRA Rank | {CONFIG['lora_r']} |
| LoRA Alpha | {CONFIG['lora_alpha']} |
| Batch Size | {CONFIG['per_device_train_batch_size']} x {CONFIG['gradient_accumulation_steps']} |
| Max Steps | {CONFIG['max_steps']} |

## License

Apache 2.0
"""

# Save model card - FIX: use MERGED_DIR not MERGED_PATH
readme_path = os.path.join(MERGED_DIR, "README.md")
with open(readme_path, "w", encoding="utf-8") as f:
    f.write(model_card)

print("✅ Model card created!")

In [ ]:
# ============================================
# UPLOAD TO HUGGINGFACE (with custom code)
# ============================================
from huggingface_hub import HfApi

api = HfApi(token=CONFIG["hf_token"])

print(f"Uploading to {CONFIG['hf_repo']}...")

# 1. Upload model files
api.upload_folder(
    folder_path=MERGED_DIR,
    repo_id=CONFIG["hf_repo"],
    repo_type="model",
    commit_message="Upload Belle-VLM model weights",
)
print("✅ Model weights uploaded!")

# 2. Create and upload custom code files for trust_remote_code
print("\nCreating custom code files...")

# configuration_llava_qwen.py
config_code = '''# Configuration for LLaVA Qwen3 model
from transformers import Qwen2Config
try:
    from transformers import Qwen3Config
    QWEN3_AVAILABLE = True
except ImportError:
    QWEN3_AVAILABLE = False
    Qwen3Config = Qwen2Config

class LlavaQwen3Config(Qwen3Config if QWEN3_AVAILABLE else Qwen2Config):
    model_type = "llava_qwen3"
    def __init__(self, mm_vision_tower=None, mm_hidden_size=None, mm_projector_type="mlp2x_gelu",
                 mm_vision_select_layer=-2, mm_vision_select_feature="patch", mm_patch_merge_type="flat",
                 mm_use_im_start_end=False, mm_use_im_patch_token=False, image_aspect_ratio="pad", **kwargs):
        super().__init__(**kwargs)
        self.mm_vision_tower = mm_vision_tower
        self.mm_hidden_size = mm_hidden_size
        self.mm_projector_type = mm_projector_type
        self.mm_vision_select_layer = mm_vision_select_layer
        self.mm_vision_select_feature = mm_vision_select_feature
        self.mm_patch_merge_type = mm_patch_merge_type
        self.mm_use_im_start_end = mm_use_im_start_end
        self.mm_use_im_patch_token = mm_use_im_patch_token
        self.image_aspect_ratio = image_aspect_ratio
'''

# Save and upload config code
with open("/tmp/configuration_llava_qwen.py", "w") as f:
    f.write(config_code)

api.upload_file(
    path_or_fileobj="/tmp/configuration_llava_qwen.py",
    path_in_repo="configuration_llava_qwen.py",
    repo_id=CONFIG["hf_repo"],
)
print("✅ Uploaded configuration_llava_qwen.py")

# modeling_llava_qwen.py (simplified version that loads mm_projector)
modeling_code = '''# LLaVA Qwen3 Model
from typing import List, Optional, Tuple, Union
from abc import ABC, abstractmethod
import os
import torch
import torch.nn as nn
from transformers import AutoConfig, AutoModelForCausalLM, Qwen2Config, Qwen2Model, Qwen2ForCausalLM
from transformers.modeling_outputs import CausalLMOutputWithPast
from transformers.generation.utils import GenerateOutput
from huggingface_hub import hf_hub_download

try:
    from transformers import Qwen3Config, Qwen3Model, Qwen3ForCausalLM
    QWEN3_AVAILABLE = True
except ImportError:
    QWEN3_AVAILABLE = False
    Qwen3Config, Qwen3Model, Qwen3ForCausalLM = Qwen2Config, Qwen2Model, Qwen2ForCausalLM

from .configuration_llava_qwen import LlavaQwen3Config

IGNORE_INDEX = -100
IMAGE_TOKEN_INDEX = -200

def build_vision_projector(config):
    projector_type = getattr(config, "mm_projector_type", "mlp2x_gelu")
    mm_hidden_size = getattr(config, "mm_hidden_size", 3072)
    hidden_size = config.hidden_size
    if projector_type == "mlp2x_gelu":
        return nn.Sequential(nn.Linear(mm_hidden_size, hidden_size), nn.GELU(), nn.Linear(hidden_size, hidden_size))
    return nn.Linear(mm_hidden_size, hidden_size)

class MobileCLIPVisionTower(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.is_loaded = False
        self.image_processor = None
        self.vision_tower = None
        self.hidden_size = getattr(config, "mm_hidden_size", 3072)
        try:
            self.image_size = int(getattr(config, "mm_vision_tower", "mobileclip_l_384").split("_")[-1])
        except:
            self.image_size = 384

    def load_model(self, **kwargs):
        if self.is_loaded:
            return
        try:
            import timm
            from transformers import CLIPImageProcessor
            self.vision_tower = timm.create_model("fastvit_mci2.apple_mclip", pretrained=True, num_classes=0)
            self.vision_tower.eval()
            self.image_processor = CLIPImageProcessor(size={"shortest_edge": self.image_size},
                crop_size={"height": self.image_size, "width": self.image_size}, do_normalize=True,
                image_mean=[0.48145466, 0.4578275, 0.40821073], image_std=[0.26862954, 0.26130258, 0.27577711])
            self.is_loaded = True
            print(f"MobileCLIP loaded!")
        except Exception as e:
            print(f"Could not load MobileCLIP: {e}")
            self.is_loaded = True

    def forward(self, images):
        if not self.is_loaded:
            self.load_model()
        with torch.no_grad():
            features = self.vision_tower.forward_features(images) if hasattr(self.vision_tower, "forward_features") else self.vision_tower(images)
        if features.dim() == 4:
            B, C, H, W = features.shape
            features = features.flatten(2).transpose(1, 2)
        elif features.dim() == 2:
            features = features.unsqueeze(1)
        return features

    def to(self, *args, **kwargs):
        super().to(*args, **kwargs)
        if self.vision_tower is not None:
            self.vision_tower = self.vision_tower.to(*args, **kwargs)
        return self

class LlavaMetaModel:
    def __init__(self, config):
        super().__init__(config)
        if hasattr(config, "mm_vision_tower") and config.mm_vision_tower:
            self.vision_tower = MobileCLIPVisionTower(config)
            self.mm_projector = build_vision_projector(config)

    def get_vision_tower(self):
        return getattr(self, "vision_tower", None)

    def load_mm_projector(self, model_path):
        """Load mm_projector weights from file or HuggingFace."""
        try:
            if os.path.isdir(model_path):
                projector_path = os.path.join(model_path, "mm_projector.bin")
            else:
                projector_path = hf_hub_download(repo_id=model_path, filename="mm_projector.bin")
            
            if os.path.exists(projector_path):
                weights = torch.load(projector_path, map_location="cpu")
                # Load weights into mm_projector
                projector_state = {}
                for k, v in weights.items():
                    if "mm_projector" in k:
                        new_key = k.replace("model.mm_projector.", "").replace("mm_projector.", "")
                        projector_state[new_key] = v
                self.mm_projector.load_state_dict(projector_state, strict=False)
                print(f"Loaded mm_projector from {projector_path}")
        except Exception as e:
            print(f"Could not load mm_projector: {e}")

class LlavaMetaForCausalLM(ABC):
    @abstractmethod
    def get_model(self): pass

    def get_vision_tower(self):
        return self.get_model().get_vision_tower()

    def encode_images(self, images):
        vt = self.get_model().get_vision_tower()
        if not vt.is_loaded:
            vt.load_model()
            vt = vt.to(device=images.device, dtype=images.dtype)
        features = vt(images)
        return self.get_model().mm_projector(features)

    def prepare_inputs_labels_for_multimodal(self, input_ids, position_ids, attention_mask, past_key_values, labels, images, image_sizes=None):
        vt = self.get_vision_tower()
        if vt is None or images is None or input_ids.shape[1] == 1:
            return input_ids, position_ids, attention_mask, past_key_values, None, labels
        
        image_features = self.encode_images(images) if images.ndim == 4 else self.encode_images(torch.cat([img for img in images], dim=0))
        if images.ndim == 5 or isinstance(images, list):
            image_features = [x.flatten(0, 1) for x in torch.split(image_features, [img.shape[0] for img in images], dim=0)]

        _labels, _position_ids, _attention_mask = labels, position_ids, attention_mask
        attention_mask = torch.ones_like(input_ids, dtype=torch.bool) if attention_mask is None else attention_mask.bool()
        position_ids = torch.arange(0, input_ids.shape[1], dtype=torch.long, device=input_ids.device) if position_ids is None else position_ids
        labels = torch.full_like(input_ids, IGNORE_INDEX) if labels is None else labels

        input_ids = [cur[mask] for cur, mask in zip(input_ids, attention_mask)]
        labels = [cur[mask] for cur, mask in zip(labels, attention_mask)]

        new_embeds, new_labels = [], []
        cur_img_idx = 0
        for batch_idx, cur_ids in enumerate(input_ids):
            num_imgs = (cur_ids == IMAGE_TOKEN_INDEX).sum()
            if num_imgs == 0:
                new_embeds.append(self.get_model().embed_tokens(cur_ids))
                new_labels.append(labels[batch_idx])
                cur_img_idx += 1
                continue
            
            img_indices = [-1] + torch.where(cur_ids == IMAGE_TOKEN_INDEX)[0].tolist() + [cur_ids.shape[0]]
            cur_embeds, cur_labels_new = [], []
            for i in range(len(img_indices) - 1):
                cur_embeds.append(self.get_model().embed_tokens(cur_ids[img_indices[i]+1:img_indices[i+1]]))
                cur_labels_new.append(labels[batch_idx][img_indices[i]+1:img_indices[i+1]])
                if i < num_imgs:
                    img_feat = image_features[cur_img_idx] if isinstance(image_features, list) else image_features[cur_img_idx:cur_img_idx+1].squeeze(0)
                    cur_img_idx += 1
                    cur_embeds.append(img_feat.to(cur_embeds[-1].dtype))
                    cur_labels_new.append(torch.full((img_feat.shape[0],), IGNORE_INDEX, device=labels[batch_idx].device, dtype=labels[batch_idx].dtype))
            new_embeds.append(torch.cat(cur_embeds))
            new_labels.append(torch.cat(cur_labels_new))

        max_len = max(x.shape[0] for x in new_embeds)
        batch_size = len(new_embeds)
        padded_embeds = []
        padded_labels = torch.full((batch_size, max_len), IGNORE_INDEX, dtype=new_labels[0].dtype, device=new_labels[0].device)
        new_attn = torch.zeros((batch_size, max_len), dtype=torch.bool, device=input_ids[0].device)
        new_pos = torch.zeros((batch_size, max_len), dtype=torch.long, device=input_ids[0].device)

        for i, (emb, lab) in enumerate(zip(new_embeds, new_labels)):
            cur_len = emb.shape[0]
            padded_embeds.append(torch.cat([emb, torch.zeros((max_len - cur_len, emb.shape[1]), dtype=emb.dtype, device=emb.device)], dim=0))
            padded_labels[i, :cur_len] = lab
            new_attn[i, :cur_len] = True
            new_pos[i, :cur_len] = torch.arange(cur_len, device=emb.device)

        return None, new_pos if _position_ids is not None else None, new_attn.to(_attention_mask.dtype) if _attention_mask is not None else None, past_key_values, torch.stack(padded_embeds), padded_labels if _labels is not None else None

class LlavaQwen3Model(LlavaMetaModel, Qwen3Model):
    config_class = LlavaQwen3Config
    def __init__(self, config):
        super().__init__(config)

class LlavaQwen3ForCausalLM(Qwen3ForCausalLM, LlavaMetaForCausalLM):
    config_class = LlavaQwen3Config
    def __init__(self, config):
        super(Qwen3ForCausalLM, self).__init__(config)
        self.model = LlavaQwen3Model(config)
        self.vocab_size = config.vocab_size
        self.lm_head = nn.Linear(config.hidden_size, config.vocab_size, bias=False)
        self.post_init()

    def get_model(self):
        return self.model

    def forward(self, input_ids=None, attention_mask=None, position_ids=None, past_key_values=None, inputs_embeds=None, labels=None, use_cache=None, output_attentions=None, output_hidden_states=None, images=None, image_sizes=None, return_dict=None, **kwargs):
        if inputs_embeds is None:
            input_ids, position_ids, attention_mask, past_key_values, inputs_embeds, labels = self.prepare_inputs_labels_for_multimodal(input_ids, position_ids, attention_mask, past_key_values, labels, images, image_sizes)
        return super().forward(input_ids=input_ids, attention_mask=attention_mask, position_ids=position_ids, past_key_values=past_key_values, inputs_embeds=inputs_embeds, labels=labels, use_cache=use_cache, output_attentions=output_attentions, output_hidden_states=output_hidden_states, return_dict=return_dict)

    @torch.no_grad()
    def generate(self, inputs=None, images=None, image_sizes=None, **kwargs):
        position_ids = kwargs.pop("position_ids", None)
        attention_mask = kwargs.pop("attention_mask", None)
        if images is not None:
            inputs, position_ids, attention_mask, _, inputs_embeds, _ = self.prepare_inputs_labels_for_multimodal(inputs, position_ids, attention_mask, None, None, images, image_sizes)
        else:
            inputs_embeds = self.get_model().embed_tokens(inputs)
        return super().generate(position_ids=position_ids, attention_mask=attention_mask, inputs_embeds=inputs_embeds, **kwargs)

    def prepare_inputs_for_generation(self, input_ids, past_key_values=None, inputs_embeds=None, **kwargs):
        images = kwargs.pop("images", None)
        image_sizes = kwargs.pop("image_sizes", None)
        inputs = super().prepare_inputs_for_generation(input_ids, past_key_values=past_key_values, inputs_embeds=inputs_embeds, **kwargs)
        if images is not None:
            inputs["images"] = images
        if image_sizes is not None:
            inputs["image_sizes"] = image_sizes
        return inputs

AutoConfig.register("llava_qwen3", LlavaQwen3Config)
AutoModelForCausalLM.register(LlavaQwen3Config, LlavaQwen3ForCausalLM)
'''

with open("/tmp/modeling_llava_qwen.py", "w") as f:
    f.write(modeling_code)

api.upload_file(
    path_or_fileobj="/tmp/modeling_llava_qwen.py",
    path_in_repo="modeling_llava_qwen.py",
    repo_id=CONFIG["hf_repo"],
)
print("✅ Uploaded modeling_llava_qwen.py")

# 3. Update config.json with auto_map
import json

config_path = os.path.join(MERGED_DIR, "config.json")
with open(config_path, "r") as f:
    config_data = json.load(f)

config_data["auto_map"] = {
    "AutoConfig": "configuration_llava_qwen.LlavaQwen3Config",
    "AutoModelForCausalLM": "modeling_llava_qwen.LlavaQwen3ForCausalLM"
}
config_data["architectures"] = ["LlavaQwen3ForCausalLM"]

with open(config_path, "w") as f:
    json.dump(config_data, f, indent=2)

api.upload_file(
    path_or_fileobj=config_path,
    path_in_repo="config.json",
    repo_id=CONFIG["hf_repo"],
)
print("✅ Updated and uploaded config.json")

print(f"\n{'='*50}")
print(f"✅ Model uploaded successfully!")
print(f"🔗 https://huggingface.co/{CONFIG['hf_repo']}")
print(f"{'='*50}")

## 8. Test Inference (From HuggingFace)

This section loads the model from HuggingFace and runs inference.

In [ ]:
# ============================================
# INFERENCE FROM HUGGINGFACE MODEL
# ============================================

import sys
import torch
from PIL import Image
from transformers import AutoTokenizer, AutoModelForCausalLM, CLIPImageProcessor

# Add ml-fastvlm to path
sys.path.insert(0, '/kaggle/working/ml-fastvlm-v2')

from llava.conversation import conv_templates
from llava.mm_utils import tokenizer_image_token, process_images
from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN

# ============================================
# CONFIG
# ============================================
HF_MODEL_PATH = "beyoru/Belle-VLM"  # Your HuggingFace model

print(f"Loading model from: {HF_MODEL_PATH}")

# ============================================
# Load Tokenizer
# ============================================
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    HF_MODEL_PATH,
    trust_remote_code=True,
    use_fast=False
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"✅ Tokenizer loaded! Vocab size: {tokenizer.vocab_size}")

In [ ]:
# ============================================
# Load Model from HuggingFace
# ============================================
from llava.model.language_model.llava_qwen import LlavaQwen3ForCausalLM

print("Loading model...")
model = LlavaQwen3ForCausalLM.from_pretrained(
    HF_MODEL_PATH,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
)
model.eval()
print(f"✅ Model loaded! Type: {model.config.model_type}")

# ============================================
# Load/Initialize Vision Tower & Image Processor
# ============================================
print("Setting up vision tower...")

# Initialize vision tower if needed
vision_tower = model.get_vision_tower()

if vision_tower is None:
    print("Vision tower not found in model, initializing...")
    # Initialize vision tower from config
    from llava.model.multimodal_encoder.builder import build_vision_tower
    vision_tower = build_vision_tower(model.config, delay_load=False)
    model.model.vision_tower = vision_tower

# Load vision tower if not already loaded
if not vision_tower.is_loaded:
    print("Loading vision tower weights...")
    vision_tower.load_model()

# Move to device
vision_tower = vision_tower.to(device=model.device, dtype=torch.float16)

# Get image processor
image_processor = vision_tower.image_processor

# Fallback if image_processor is still None
if image_processor is None:
    print("⚠️ image_processor is None, using default CLIPImageProcessor...")
    image_processor = CLIPImageProcessor.from_pretrained("openai/clip-vit-large-patch14-336")
    # Override with correct image size for mobileclip_l_384
    image_processor.size = {"shortest_edge": 384}
    image_processor.crop_size = {"height": 384, "width": 384}

print(f"✅ Vision tower ready!")
print(f"   Image processor: {type(image_processor).__name__}")
print(f"   Image size: {getattr(image_processor, 'size', 'unknown')}")

In [ ]:
# ============================================
# TEST INFERENCE
# ============================================
from datasets import load_dataset
from IPython.display import display

# Load test data
print("Loading test dataset...")
test_dataset = load_dataset("5CD-AI/Viet-multimodal-open-r1-8k-verified", split="train")

test_image = test_dataset[0]['image']
test_question = test_dataset[0]['vi_problem']
expected_answer = test_dataset[0]['vi_solution']

# Display
print("Test image:")
display(test_image.resize((300, 300)))
print(f"\nQuestion: {test_question[:200]}...")

# Prepare image
if test_image.mode != 'RGB':
    test_image = test_image.convert('RGB')

image_tensor = process_images([test_image], image_processor, model.config)[0]

# Build prompt
conv = conv_templates["qwen_3"].copy()
conv.append_message(conv.roles[0], f"{DEFAULT_IMAGE_TOKEN}\n{test_question}")
conv.append_message(conv.roles[1], None)
prompt = conv.get_prompt()

# Tokenize
input_ids = tokenizer_image_token(
    prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt'
).unsqueeze(0).to(model.device)

print(f"\nInput shape: {input_ids.shape}")
print("Generating response...")

# Generate
with torch.inference_mode():
    output_ids = model.generate(
        input_ids,
        images=image_tensor.unsqueeze(0).to(dtype=torch.float16, device=model.device),
        image_sizes=[test_image.size],
        do_sample=True,
        temperature=0.7,
        top_p=0.8,
        max_new_tokens=512,
        use_cache=True,
        pad_token_id=tokenizer.pad_token_id,
    )

# Decode
output = tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0].strip()

# Results
print("=" * 60)
print("INFERENCE RESULTS")
print("=" * 60)
print(f"\nQuestion:\n{test_question[:300]}...")
print(f"\nModel Response:\n{output}")
print(f"\nExpected:\n{expected_answer[:300]}...")
print("=" * 60)

In [ ]:
# ============================================
# HELPER FUNCTION FOR EASY INFERENCE
# ============================================

def ask_vlm(image, question, max_tokens=512, temperature=0.7):
    """
    Simple inference function for Belle-VLM.
    
    Args:
        image: PIL Image or path to image
        question: Question about the image (Vietnamese or English)
        max_tokens: Maximum tokens to generate
        temperature: Sampling temperature
    
    Returns:
        Model response string
    """
    from PIL import Image as PILImage
    
    # Load image if path
    if isinstance(image, str):
        image = PILImage.open(image)
    
    # Convert to RGB
    if image.mode != 'RGB':
        image = image.convert('RGB')
    
    # Process image
    img_tensor = process_images([image], image_processor, model.config)[0]
    
    # Build prompt
    conv = conv_templates["qwen_3"].copy()
    conv.append_message(conv.roles[0], f"{DEFAULT_IMAGE_TOKEN}\n{question}")
    conv.append_message(conv.roles[1], None)
    prompt = conv.get_prompt()
    
    # Tokenize
    input_ids = tokenizer_image_token(
        prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt'
    ).unsqueeze(0).to(model.device)
    
    # Generate
    with torch.inference_mode():
        output_ids = model.generate(
            input_ids,
            images=img_tensor.unsqueeze(0).to(dtype=torch.float16, device=model.device),
            image_sizes=[image.size],
            do_sample=True if temperature > 0 else False,
            temperature=temperature,
            top_p=0.8,
            max_new_tokens=max_tokens,
            use_cache=True,
            pad_token_id=tokenizer.pad_token_id,
        )
    
    return tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0].strip()


# Test the helper function
print("Testing helper function...")
response = ask_vlm(test_image, "Mô tả hình ảnh này.")
print(f"Response: {response}")

## Done!

The model has been:
1. ✅ Trained on Vietnamese multimodal dataset
2. ✅ Merged LoRA weights
3. ✅ Pushed to HuggingFace
4. ✅ Tested with inference

### View your model
🔗 https://huggingface.co/beyoru/Belle-VLM

### Quick Usage

```python
# Use the ask_vlm function
response = ask_vlm(your_image, "Mô tả hình ảnh này.")
print(response)

# Or with custom parameters
response = ask_vlm(
    image="path/to/image.jpg",
    question="Trong hình có gì?",
    max_tokens=1024,
    temperature=0.5
)
```